<a href="https://colab.research.google.com/github/enf3tri/machine-learning/blob/main/Conv3D%2BFullyConn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install keras-metrics

In [ ]:
import keras
from keras import applications
from keras.preprocessing.image import ImageDataGenerator
from keras import optimizers
from keras.models import Sequential, Model 
from keras.layers import *
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
 
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import keras_metrics as km
import pandas as pd 
 
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import multilabel_confusion_matrix
from tensorflow.python.keras.applications.resnet import ResNet50
from sklearn.metrics import classification_report
from keras.applications.resnet import ResNet50

import sys
sys.path.insert(0,'/content/drive/MyDrive/fight-detection-surv-dataset-master/')
classes = ["noFight","fight","noFight","fight","noFight","fight"]

In [ ]:
seq_len = 70 
img_height , img_width = 64, 64

In [ ]:
fight_dataset = pd.read_csv('/content/drive/MyDrive/fight-detection-surv-dataset-master/fight-set.csv')
no_fight_dataset = pd.read_csv('/content/drive/MyDrive/fight-detection-surv-dataset-master/no-fight-set.csv')
all_dataset = pd.read_csv('/content/drive/MyDrive/fight-detection-surv-dataset-master/all-set.csv')
label_dataset = pd.read_csv('/content/drive/MyDrive/fight-detection-surv-dataset-master/label-set.csv')

data_path = "content/drive/MyDrive/fight-detection-surv-dataset-master/data/"


train_set = pd.read_csv('/content/drive/MyDrive/fight-detection-surv-dataset-master/train-set.csv')
test_set = pd.read_csv('/content/drive/MyDrive/fight-detection-surv-dataset-master/test-set.csv')


In [ ]:
def frames_extraction(video_path):
    frames_list = []
     
    vidObj = cv2.VideoCapture(video_path)
    # Used as counter variable 
    count = 1
 
    while count <= seq_len: 
         
        success, image = vidObj.read() 
        if success:
            image = cv2.resize(image, (img_height, img_width))
            frames_list.append(image)
            count += 1
        else:
            print("Defected frame")
            break
 
            
    return frames_list

In [ ]:
def create_data(input_dir):
    X = []
    Y = []
     
    classes_list = os.listdir(input_dir)
     
    for c in classes_list:
        print(c)
        files_list = os.listdir(os.path.join(input_dir, c))
        for f in files_list:
           frames = frames_extraction(os.path.join(os.path.join(input_dir, c), f))
           if len(frames) == seq_len:
                X.append(frames)
             
                y = [0]*len(classes)
                y[classes.index(c)] = 1
                Y.append(y)
     
    X = np.asarray(X)
    Y = np.asarray(Y)
    return X, Y

In [ ]:
X, Y = create_data(data_path)
 
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.30, shuffle=True, random_state=0)
 
model = Sequential()
model.add(Conv3D(32, kernel_size=(3, 3, 1), activation='relu', input_shape=input_shape))
model.add(Conv3D(32, kernel_size=(3, 3, 1), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 1)))

model.add(Dropout(0.4))
model.add(Conv3D(64, kernel_size=(3, 3, 1), activation='relu'))
model.add(Conv3D(64, kernel_size=(3, 3, 1), activation='relu'))
model.add(MaxPooling3D(pool_size=(2, 2, 1)))
model.add(Dropout(0.4))
model.add(Dense(128))
model.add(Dropout(0.4))

model.add(Dense(128))
model.add(Dropout(0.4))
model.add(Dense(256, activation='relu'))
model.add(Flatten())

model.add(Dense(10, activation='softmax'))

 
model.summary()
 
opt = keras.optimizers.SGD(lr=0.001)
model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=["accuracy"])
 
earlystop = EarlyStopping(patience=7)
callbacks = [earlystop]
 
history = model.fit(x = X_train, y = y_train, epochs=40, batch_size = 8 , shuffle=True, validation_split=0.2, callbacks=callbacks)
 
y_pred = np.argmax(y_pred, axis = 1)
y_test = np.argmax(y_test, axis = 1)
 
print(classification_report(y_test, y_pred))

FileNotFoundError: ignored